# Zadanie

### Wstęp 

Celem projektu jest oddtworzenie zadanych poleceń z sql za pomocą funkcji i metod z pakietu pandas, a także upewnienie się, że uzyskane przez nas wyniki są tożsame, co sprawdzimy przy pomocy dostępnej w pakiecie pandas metody .equals().


In [64]:
import pandas as pd
import numpy as np
import os, os.path
import sqlite3
import tempfile

baza = os.path.join(tempfile.mkdtemp(), 'baza.db')
if os.path.isfile(baza):
    os.remove(baza) 

conn = sqlite3.connect(baza)


Tags = pd.read_csv("Tags.csv")
Posts = pd.read_csv("Posts.csv")
Comments = pd.read_csv("Comments.csv")
Users = pd.read_csv("Users.csv")

Tags.to_sql("Tags", conn)
Posts.to_sql("Posts", conn)
Comments.to_sql("Comments", conn)
Users.to_sql("Users", conn)

### Przykład 1

Z tabeli Tags należy wybrać kolumny TagsName i Count. Następnie należy ustawić wiersze malejąco pod względem wartości z kolumny Count i wybrać pierwsze 10 wierszy.

1. Rozwiązanie referencyjne

In [65]:
df_sql_1 = pd.read_sql_query("""SELECT TagName, Count
                  FROM Tags
                  ORDER BY Count DESC
                  LIMIT 10""", conn)

2. Rowiązanie za pomocą funkcji i metod z pakietu pandas

In [66]:
df_1 = Tags[["TagName", "Count"]].sort_values("Count", ascending = False)\
    .iloc[0:10].reset_index(drop = True)

df_1

,TagName,Count
0,visas,5271
1,usa,2858
2,air-travel,2830
3,uk,2114
4,schengen,2094
5,customs-and-immigration,1798
6,transit,1204
7,trains,1031
8,passports,954
9,indian-citizens,916


* Sprawdzenie tożsamości uzyskanych wyników

In [67]:
df_1.equals(df_sql_1)

True

### Przykład 2

Tworzymy tabelkę CmtTotScr poprzez przypisanie każdej występującej w tabelce Comments kombinacji wartości PostId i i UserId sumy wartości Score w kolumnie, którą nazywamy CommentsTotalScore.

Łączymy tabelkę Posts z CmtTotScr poprzez wartości z kolumny Posts.Id i wartości z kolumny CmTotScr.CommentsTotalScore - łączymy rzędy Posts z rzędami CmTotScr tam, gdzie wartość Posts.Id jest równa wartości CmTotScr. CommentsTotalScore. Wybieramy wiersze, w których wartość w kolumnie Posts.PostTypeId jest równa 1. Następnie ustawiamy wiersze w kolejności malejącej pod względem wartości z CommentsTotalScore i wybieramy pierwsze 10 wierszy. Na koniec wybieramy kolumny Posts.Title, Posts.CommentCount, CmTotScr.CommentsTotalScore i Posts.ViewCount.

1. Rozwiązanie Referencyjne

In [68]:
df_sql_2 = pd.read_sql_query("""SELECT
                          Posts.Title, Posts.CommentCount,
                          CmtTotScr.CommentsTotalScore,
                          Posts.ViewCount
                          FROM (SELECT PostID,UserID,
                              SUM(Score) AS CommentsTotalScore
                              FROM Comments
                              GROUP BY PostID, UserID
                              ) AS CmtTotScr
                          JOIN Posts ON Posts.ID=CmtTotScr.PostID
                          WHERE Posts.PostTypeId=1
                          ORDER BY CmtTotScr.CommentsTotalScore DESC
                          LIMIT 10""", conn)

2. Rowiązanie za pomocą funkcji i metod z pakietu pandas

In [69]:
tmp = Comments.groupby(["PostId", "UserId"])
CmtTotScr = pd.DataFrame([{"CommentsTotalScore" : sum(ramka.Score),
                           "PostId" : grupa[0],
                           "UserId" : grupa[1]}
                          for (grupa, ramka) in tmp])
df_2 = Posts.loc[Posts.PostTypeId == 1, ["Id", "Title", "CommentCount", "ViewCount"]]
df_2 = pd.merge(df_2, CmtTotScr, left_on = "Id", right_on = "PostId" )
df_2 = df_2[["Title", "CommentCount", "CommentsTotalScore", "ViewCount"]]\
    .sort_values("CommentsTotalScore", ascending = False).iloc[0:10]\
        .reset_index(drop = True)

df_2

,Title,CommentCount,CommentsTotalScore,ViewCount
0,"Boss is asking for passport, but it has a stam...",24,207,54982.0
1,Why don't airlines have backup planes just in ...,26,172,14516.0
2,"OK we're all adults here, so really, how on ea...",27,155,73808.0
3,How to cross a road by foot in a country that ...,25,140,5240.0
4,Where can I change my clothes at the airport?,16,128,12020.0
5,"Boss is asking for passport, but it has a stam...",24,121,54982.0
6,How to avoid toddlers on a long-distance plane...,19,120,24955.0
7,Job interview in London requires me to wire mo...,23,116,14827.0
8,What to do without underwear on a 4 day trip?,13,110,11713.0
9,"OK, we are all adults here, so what is a bidet...",28,109,52265.0


* Sprawdzenie tożsamości uzyskanych wyników

In [70]:
df_2.equals(df_sql_2)

True

### Przykład 3

Łączymy ze sobą tabelkę Users i Posts poprzez wartości z kolumny Users.AccountId i wartości z kolumny Posts.OwnerUserId - łączymy rzędy Users z rzędami Posts tam, gdzie wartość Users.AccountId jest równa wartości Posts.OwnerUsersId. Następnie usuwamy wiersze, w których wartość w kolumnie OwnerUserId wynosi -1. Każdej wartości z OwnerUserId przypisujemy średnią wartość Score W kolumnie, którą nazywamy PostsMeanScore i maksymalną wartość CreationDate w kolumnie, którą nazywamy LastPostCreationDate. Wybieramy z utworzonej tabelki kolumny Users.DisplayName, Users.Age, Users.Location, Posts.PostsMeanScore i Posts.LastPostCreationDate, a następnie ustawiamy wiersze w kolejności malejącej pod względem wartości z PostsMeanScore i wybieramy pierwsze 10 wierszy.

1. Rozwiązanie referencyjne

In [71]:
df_sql_3 = pd.read_sql_query("""SELECT Users.DisplayName, Users.Age, Users.Location,
                                AVG(Posts.Score) as PostsMeanScore,
                                MAX(Posts.CreationDate) AS LastPostCreationDate
                                FROM Posts
                                JOIN Users ON Users.AccountId=Posts.OwnerUserId
                                WHERE OwnerUserId != -1
                                GROUP BY OwnerUserId
                                ORDER BY PostsMeanScore DESC
                                LIMIT 10""", conn)

2. Rowiązanie za pomocą funkcji i metod z pakietu pandas

In [72]:
tmp1 = Posts.loc[Posts.OwnerUserId != -1].groupby("OwnerUserId").Score.agg("mean").to_frame().reset_index()
tmp2 = Posts.loc[Posts.OwnerUserId != -1].groupby("OwnerUserId").CreationDate.agg("max").to_frame().reset_index()
df_tmp = pd.merge(tmp1, tmp2, on = "OwnerUserId")

df_3 = pd.merge(Users, df_tmp, left_on = "AccountId", right_on = "OwnerUserId")
df_3 = df_3[["DisplayName", "Age", "Location", "Score", "CreationDate_y"]].sort_values("Score", ascending = False).iloc[0:10]\
        .reset_index(drop = True).rename(columns = {"Score" : "PostsMeanScore", "CreationDate_y" : "LastPostCreationDate"})

df_3

,DisplayName,Age,Location,PostsMeanScore,LastPostCreationDate
0,Oded,44.0,"London, United Kingdom",52.0,2011-11-03T14:40:36.870
1,Rook,NaN,NaN,50.0,2016-05-27T03:17:41.753
2,JPhi1618,NaN,"Dallas, Texas, United States",45.0,2015-09-26T17:43:19.090
3,csmba,42.0,"San Francisco, CA",43.0,2012-01-15T14:27:11.070
4,Petrogad,32.0,NaN,41.0,2016-04-16T19:44:21.020
5,Josh,NaN,Australia,33.0,2011-12-23T18:16:21.877
6,Dan Esparza,43.0,"Atlanta, GA, United States",30.0,2014-01-26T21:12:59.673
7,James,NaN,NaN,29.5,2017-05-11T00:13:37.897
8,Brad Rhoads,NaN,"Nampa, ID",29.0,2016-04-11T11:10:14.240
9,Dexter,34.0,"London, United Kingdom",28.5,2015-05-06T00:23:36.353


* Sprawdzenie tożsamości uzyskanych wyników

In [73]:
df_3.equals(df_sql_3)

True

### Przykład 4

Tworzymy tabelkę Tab1 poprzez połączenie tabelki Users i Posts poprzez wartości z kolumny Users.Id i wartości z kolumny Posts.OwnerUserId - łączymy rzędy Users z rzędami Posts tam, gdzie wartość Users.Id jest równa wartości Posts.OwnerUsersId. Wybieramy wiersze, w których wartość PostTypeId jest równa 2. Każdej wartości z kolumny Users.Id przypisujemy ilość jej przypisanych wartości DisplayName w kolumnie, którą nazywamy AnswersNumber. Na koniec wybieramy z tabelki kolumny Users.Id, Users.DisplayName i AnswersNumber.

Następnie tworzymy tabelkę Tab2 poprzez połączenie tabelek Users i Posts poprzez wartości z kolumny Users.Id i wartości z kolumny Posts.OwnerUserId - łączymy rzędy Users z rzędami Posts tam, gdzie wartość Users. Wybieramy wiersze, w których wartość PostTypeId jest równa 1. Każdej wartości Id przypisujemy ilość jej występowania w kolumnie, którą nazywamy QuestionsNumber. Na koniec wybieramy z tabelki kolumny Users.Id i QuestionsNumber.

Łączymy ze sobą obie tabelki poprzez kolumnę Id - łączymy rzędy Tab1 i Tab2 tam, gdzie wartość Tab1.Id jest równa Tab2.Id. Wybieramy wiersze, w których wartość w kolumnie QuestionsNumber jest mniejsza niż wartość w kolumnie AnswersNumber. Ustawiamy wiersze w kolejności malejącej pod względem wartości z Answers Number. Wybieramy z tabelki kolumny DisplayName, QuestionsNumber i AnswersNumber.

1. Rozwiązanie Referencyjne

In [99]:
df_sql_4 = pd.read_sql_query("""SELECT DisplayName, QuestionsNumber, AnswersNumber
                                FROM
                                    (SELECT COUNT(*) as AnswersNumber, Users.DisplayName, Users.Id
                                        FROM Users 
                                        JOIN Posts ON Users.Id = Posts.OwnerUserId
                                        WHERE Posts.PostTypeId = 2
                                        GROUP BY Users.Id) AS Tab1
                                JOIN
                                    (SELECT COUNT(*) as QuestionsNumber, Users.Id
                                        FROM Users JOIN Posts ON Users.Id = Posts.OwnerUserId
                                        WHERE Posts.PostTypeId = 1
                                        GROUP BY Users.Id) AS Tab2
                                ON Tab1.Id = Tab2.Id
                                WHERE QuestionsNumber < AnswersNumber
                                ORDER BY AnswersNumber DESC LIMIT 10""", conn)

2. Rowiązanie za pomocą funkcji i metod z pakietu pandas

In [100]:
tab1 = pd.merge(Users[["DisplayName", "Id"]], Posts.loc[Posts.PostTypeId == 2, ["OwnerUserId"]],
                left_on = "Id", right_on = "OwnerUserId")
tmp = tab1.groupby("Id").OwnerUserId.agg("count").to_frame().reset_index().rename(columns = {"OwnerUserId" : "AnswersNumber"})
tab1 = pd.merge(tmp, Users[["DisplayName", "Id"]], how = "left", on = "Id")

tab2 = pd.merge(Users[["Id"]], Posts.loc[Posts.PostTypeId == 1, ["OwnerUserId"]],
                left_on = "Id", right_on = "OwnerUserId")
tab2 = tab2.groupby("Id").OwnerUserId.agg("count").to_frame().reset_index().rename(columns = {"OwnerUserId" : "QuestionsNumber"})

df_4 = pd.merge(tab1, tab2, on = "Id")
df_4 = df_4.loc[df_4.QuestionsNumber < df_4.AnswersNumber, ["DisplayName", "QuestionsNumber", "AnswersNumber"]]\
        .sort_values("AnswersNumber", ascending = False).reset_index(drop = True).iloc[0:10]

df_4

,DisplayName,QuestionsNumber,AnswersNumber
0,Mark Mayo,318,1726
1,Relaxed,8,1225
2,jpatokal,58,999
3,Gayot Fow,86,824
4,Burhan Khalid,12,638
5,Karlson,44,631
6,chx,61,594
7,Crazydre,119,587
8,phoog,5,532
9,JonathanReez,204,493


* Sprawdzenie tożsamości uzyskanych wyników

In [101]:
df_4.equals(df_sql_4)

True

### Przykład 5

Tworzymy tabelkę BestAnswers poprzez wybranie z tabelki Posts wierszy, w których wartości w kolumnie PostTypeId jest równe 2. Następnie do każdej wartości w kolumnie ParentId przypisujemy maksymalną wartość Score w kolumnie, którą nazywamy MaxScore. Następnie wybieramy kolumny Id, ParentId i MaxScore.

Tworzymy tablekę Questions poprzez wybranie z tabelki Posts wierszy, w których wartość w kolumnie PostTypeId jest równa 1.

Łączymy ze sobą tabelki BestAnswers i Questions poprzez wartości z kolumny Questions.Id i wartości z kolumny BestAnswers.ParentId - łączymy rzędy BestAnswers z rzędami Questions tam, gdzie wartość Questions.Id jest równa wartości BestAnswers.ParentId. Następnie w ten sposób utworzoną tabelkę łączymy z tabelką Posts poprzez wartości z kolumny Questions.AcceptedAnswerId i wartości z kolumny Posts.Id - łączymy rzędy Posts z rzędami utworzonej tabelki tam, gdzie wartość Questions.AcceptedAnswerId jest równa wartości z Posts.Id.

Wybieramy kolumny Questions.Id, Questions.Title, BestAnswers.MaxScore, Posts.Score (zmieniamy jej nazwę na AcceptedScore), a następnie dodajemny kolumnę Difference, która zawiera wartości MaxScore - AcceptedScore. Wiersze ustawiamy malejąco pod względem wartości z kolmuny Difference i wybieramy pierwsze 10 wierszy.

1. Rozwiązanie referencyjne

In [96]:
df_sql_5 = pd.read_sql_query("""SELECT Questions.Id, Questions.Title, BestAnswers.MaxScore, 
                                Posts.Score AS AcceptedScore, BestAnswers.MaxScore-Posts.Score AS Difference
                                FROM 
                                    (SELECT Id, ParentId, MAX(Score) AS MaxScore
                                    FROM Posts
                                    WHERE PostTypeId==2
                                    GROUP BY ParentId) AS BestAnswers
                                JOIN 
                                    (SELECT * FROM Posts
                                    WHERE PostTypeId==1) AS Questions
                                ON Questions.Id=BestAnswers.ParentId
                                JOIN Posts ON Questions.AcceptedAnswerId=Posts.Id
                                ORDER BY Difference DESC
                                LIMIT 10""", conn)

2. Rowiązanie za pomocą funkcji i metod z pakietu pandas

In [97]:
BestAnswers = Posts.loc[Posts.PostTypeId == 2, ["ParentId", "Score"]].groupby("ParentId").Score.agg("max").to_frame()\
                .reset_index().rename(columns = {"Score" : "MaxScore"})
df_5 = pd.merge(Posts.loc[Posts.PostTypeId == 1, ["Id", "Title", "AcceptedAnswerId"]], BestAnswers,
                left_on = "Id", right_on = "ParentId")

df_5 = pd.merge(df_5[["Id", "Title", "AcceptedAnswerId", "MaxScore"]], Posts[["Id", "Score"]], 
               left_on = "AcceptedAnswerId", right_on = "Id")

df_5 = df_5.assign(Difference = (df_5.MaxScore - df_5.Score))[["Id_x", "Title", "MaxScore", "Score", "Difference"]]\
            .rename(columns = {"Id_x" : "Id", "Score" : "AcceptedScore"}).sort_values("Difference", ascending = False)\
            .iloc[0:10].reset_index(drop = True)

df_5

,Id,Title,MaxScore,AcceptedScore,Difference
0,99713,"In Germany, what are you supposed to do if you...",126,33,93
1,81376,What is way to eat rice with hands in front of...,120,30,90
2,76737,Why do many countries in the world still requi...,118,31,87
3,38177,How do you know if Americans genuinely/literal...,245,166,79
4,90636,Caught speeding 111 Mph (179 km/h) in Californ...,101,25,76
5,81492,Why is international first class much more exp...,90,21,69
6,94320,Strange looking region of France on Google Maps,71,7,64
7,80050,What's an easy way of making my luggage unique...,72,16,56
8,69224,How can a tourist obtain an 'authentic' duelli...,96,47,49
9,20171,How to prevent the passenger in front of you f...,59,12,47


* Sprawdzenie tożsamości uzyskanych wyników

In [98]:
df_5.equals(df_sql_5)

True

In [102]:
conn.close()